In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import pycountry

In [2]:
dfs = pd.read_html('pwc_rates.html')

In [31]:
df = dfs[0]
# remove all bracketed text
df = df.replace(to_replace='\(.*\)', value='', regex=True)
df.columns = ['country', 'full']

def get_iso2(country):
    manual = {
        'Namibia, Republic of': 'NA',
        'Macau SAR': 'MO',
        'Cameroon, Republic of': 'CM',
        "China, People's Republic of": 'CN',
        'Hong Kong SAR': 'HK',
        'Uzbekistan, Republic of': 'UZ',
        'Congo, Democratic Republic of the': 'CD',
        'Congo, Republic of' : 'CG',
        'Guernsey, Channel Islands': 'GG',
        'Ivory Coast': 'CI',
        'Jersey, Channel Islands': 'JE',
        'Lao PDR': 'LA',
        'Palestinian territories': 'PS',

    }
    country = country.strip()
    if country in manual:
        return manual[country]
    try:
        return pycountry.countries.get(name=country).alpha_2
    except:
        # try a fuzzy match
        try:
            fuzzy = pycountry.countries.search_fuzzy(country)[0].alpha_2
            return fuzzy
        except:
            print(f'    ❗️Could not find fuzzy match for {country}')
            return None
        return None
    
df['id'] = df['country'].apply(get_iso2)
df['value'] = pd.to_numeric(df['full'], errors='coerce')/100

df['numeric'] = df.id.apply(lambda x: pycountry.countries.get(alpha_2=x).numeric)

df.to_csv('pwc_rates_with_numeric.csv', index=False)

In [33]:
df

,country,full,id,value,numeric
0,Albania,20,AL,0.20,008
1,Algeria,19,DZ,0.19,012
2,Angola,14,AO,0.14,024
3,Argentina,21,AR,0.21,032
4,Armenia,20,AM,0.20,051
...,...,...,...,...,...
145,"Uzbekistan, Republic of",12,UZ,0.12,860
146,Venezuela,16,VE,0.16,862
147,Vietnam,10,VN,0.10,704
148,Zambia,16,ZM,0.16,894


In [11]:
pycountry.countries.get(name="Albania").alpha_2

'AL'